In [289]:
class Flipflop:
    stored = 0
    def __init__(self, destinations, name) -> None:
        self.destinations = destinations
        self.name = name
        pass
    
    def send_pulse(self, input, source, queue):
        if not input:
            self.stored = (self.stored + 1) % 2
            for destination in self.destinations:
                queue.append((destination, self.stored, self.name))
        return None

In [290]:
class Broadcaster:
    def __init__(self, destinations) -> None:
        self.destinations = destinations
        pass

In [291]:
class Conjunction:
    def __init__(self, destinations, name) -> None:
        self.destinations = destinations
        self.sources = []
        self.name = name
        self.memory = {}
        pass

    def add_source(self, source):
        self.sources.append(source)
        self.memory[source] = 0
    
    def send_pulse(self, input, source, queue):
        self.memory[source] = input
        if all([x == 1 for x in self.memory.values()]):
            value = 0
        else:
            value = 1
        for destination in self.destinations:
            queue.append((destination, value, self.name))

In [292]:
text = open("input.txt").read().split("\n")
text = [x.split(" -> ") for x in text]

modules = {}

for module in text:
    if module[0] == 'broadcaster':
        broadcaster = Broadcaster(module[1].split(", "))
    elif module[0][0] == '%':
        modules[module[0][1:]] = Flipflop(module[1].split(", "), module[0][1:])
    elif module[0][0] == '&':
        modules[module[0][1:]] = Conjunction(module[1].split(", "), module[0][1:])

for module in modules:
    flipflop = modules[module]
    for destination in flipflop.destinations:
        if destination in modules and isinstance(modules[destination], Conjunction):
            modules[destination].add_source(module)

modules['rx'] = Flipflop([], "rx")

In [293]:
count = 0
limit = 1000
queue = []
low_count = 0
high_count = 0
while(True):
    # print(count, queue, high_count, low_count)
    if len(queue) == 0:
        if count >= limit:
            break
        count += 1
        low_count += 1
        for destination in broadcaster.destinations:
            low_count += 1
            modules[destination].send_pulse(0, 'broadcaster', queue)
    else:
        next_module, pulse, source = queue.pop(0)
        if pulse:
            high_count += 1
        else:
            low_count += 1
        modules[next_module].send_pulse(pulse, source, queue)
print(low_count, high_count, low_count * high_count)

18548 45383 841763884
